# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle
nltk.download('punkt')
nltk.download('wordnet')
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql ('SELECT * FROM tech', engine)
X = df ['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    pass
#converting all the text to lowercase
    text.lower()
    # removing punctuation characters from the text and replacing them with an empty space
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # use the word tokenizer to convert text into tokens
    tokens = word_tokenize(text)
    # removing stopwords by calling a "for loop" for the tokens and using english stopwords from nltk!
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    # Using lemmatization to strip all the words
    tokens = [WordNetLemmatizer().lemmatize(word).strip() for word in tokens]
    # finally return the tokens
    return tokens

print(X[3])
print(tokenize(X[3]))

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['UN', 'report', 'Leogane', '80', '90', 'destroyed', 'Only', 'Hospital', 'St', 'Croix', 'functioning', 'Needs', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
 # train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

In [7]:
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)


related
             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1530
          1       0.85      0.91      0.88      4985
          2       0.29      0.31      0.30        39

avg / total       0.79      0.81      0.80      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5431
          1       0.80      0.42      0.55      1123

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.79      3810
          1       0.74      0.60      0.66      2744

avg / total       0.74      0.74      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [9]:
#parameters = {
        #'vect__max_df':[0.75,1.0],
        #'clf__estimator__n_estimators': [20, 50]
   # }

#cv = GridSearchCV(pipeline, param_grid=parameters)
#cv.fit(X_train, Y_train)

In [10]:
parameters = { 'vect__max_df': (0.75, 1.0),
             #  'vect__stop_words': ('english', None),
                'clf__estimator__n_estimators': [10, 20],
                'clf__estimator__min_samples_split': [2, 5]
              }

cv = GridSearchCV (pipeline, param_grid= parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)


related
             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1530
          1       0.85      0.91      0.88      4985
          2       0.29      0.31      0.30        39

avg / total       0.79      0.81      0.80      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5431
          1       0.80      0.42      0.55      1123

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.79      3810
          1       0.74      0.60      0.66      2744

avg / total       0.74      0.74      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



shelter
             precision    recall  f1-score   support

          0       0.94      0.99      0.97      5943
          1       0.85      0.37      0.51       611

avg / total       0.93      0.93      0.92      6554

clothing
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6453
          1       0.53      0.10      0.17       101

avg / total       0.98      0.98      0.98      6554

money
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      6382
          1       0.50      0.01      0.01       172

avg / total       0.96      0.97      0.96      6554

missing_people
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6485
          1       1.00      0.01      0.03        69

avg / total       0.99      0.99      0.98      6554

refugees
             precision    recall  f1-score   support

          0       0.97      1.00      

In [12]:
model = cv
print(model)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.75, 1.0), 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


In [13]:
#model.fit(X_train, Y_train)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
pickle.dump(model, open('classifier.pkl', 'wb'))

In [16]:
#Load the model from pickle file
#model = pickle.load(open(classifier.pkl, 'rb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.